Kaggle intro

In [1]:
# https://cloud.google.com/storage/docs/reference/libraries

def create_bucket():
    # Imports the Google Cloud client library
    from google.cloud import storage

    # Instantiates a client
    storage_client = storage.Client()

    # The name for the new bucket
    bucket_name = 'my-first-bucket-max-gonchar-oct-25'

    # Creates the new bucket
    bucket = storage_client.create_bucket(bucket_name)

    print('Bucket {} created.'.format(bucket.name))
    
    
# create_bucket()

In [2]:
# https://cloud.google.com/docs/authentication/getting-started#auth-cloud-implicit-python

def list_buckets():
    from google.cloud import storage

    # If you don't specify credentials when constructing the client, the
    # client library will look for credentials in the environment.
    storage_client = storage.Client()

    # Make an authenticated API request
    buckets = list(storage_client.list_buckets())
    print(buckets)

    
list_buckets()

[<Bucket: my-first-bucket-max-gonchar-oct-25>]


In [3]:
from google.cloud import bigquery

In [4]:
# Client object - a central role in retrieving information from BigQuery datasets

# Client objects hold projects and a connection to the BigQuery service

client = bigquery.Client()

In [5]:
# bigquery-public-data is a project
# A project is a collection of datasets

# hacker-news is a dataset
# A dataset is a collection of tables

# Construct a reference to the "hacker_news" dataset
dataset_ref = client.dataset('hacker_news', project='bigquery-public-data')

# Do an API request to fetch the dataset
hacker_news_dataset = client.get_dataset(dataset_ref)

In [6]:
# List tables in the dataset
# Dataset is like a spreadsheet file containing multiple tables

dataset_tables = list(client.list_tables(hacker_news_dataset))

for table in dataset_tables:
    display(table.table_id)

'comments'

'full'

'full_201510'

'stories'

In [7]:
# Fetch table
comments_table_ref = dataset_ref.table('comments')
table = client.get_table(comments_table_ref)

In [8]:
# Overview table schema

# Column == Field

# SchemaField format:
# name of the column
# mode of the column (NULLABLE - allows NULL values)
# description

display(table.schema)

[SchemaField('id', 'INTEGER', 'NULLABLE', 'Unique comment ID', ()),
 SchemaField('by', 'STRING', 'NULLABLE', 'Username of commenter', ()),
 SchemaField('author', 'STRING', 'NULLABLE', 'Username of author', ()),
 SchemaField('time', 'INTEGER', 'NULLABLE', 'Unix time', ()),
 SchemaField('time_ts', 'TIMESTAMP', 'NULLABLE', 'Human readable time in UTC (format: YYYY-MM-DD hh:mm:ss)', ()),
 SchemaField('text', 'STRING', 'NULLABLE', 'Comment text', ()),
 SchemaField('parent', 'INTEGER', 'NULLABLE', 'Parent comment ID', ()),
 SchemaField('deleted', 'BOOLEAN', 'NULLABLE', 'Is deleted?', ()),
 SchemaField('dead', 'BOOLEAN', 'NULLABLE', 'Is dead?', ()),
 SchemaField('ranking', 'INTEGER', 'NULLABLE', 'Comment ranking', ())]

In [9]:
# Check just the first N lines of the table (to make sure the one is the right one)

# client.list_rows -> RowIterator -> pd.DataFrame

# list first 3 rows
comments_df = client.list_rows(table, max_results=3).to_dataframe()
display(comments_df)

# list 3 rows of id and by colums
author_rows_df = client.list_rows(table, selected_fields=table.schema[:2], max_results=3).to_dataframe()
display(author_rows_df)

,id,by,author,time,time_ts,text,parent,deleted,dead,ranking
0,2701393,5l,5l,1309184881,2011-06-27 14:28:01+00:00,And the glazier who fixed all the broken windo...,2701243,None,None,0
1,5811403,99,99,1370234048,2013-06-03 04:34:08+00:00,Does canada have the equivalent of H1B/Green c...,5804452,None,None,0
2,21623,AF,AF,1178992400,2007-05-12 17:53:20+00:00,"Speaking of Rails, there are other options in ...",21611,None,None,0


,id,by
0,2701393,5l
1,5811403,99
2,21623,AF


In [11]:
chicago_dataset_ref = client.dataset('chicago_crime', project='bigquery-public-data')
chicago_dataset = client.get_dataset(chicago_dataset_ref)

In [19]:
chicago_dataset_tables = list(client.list_tables(chicago_dataset_ref))
for table in chicago_dataset_tables:
    display(table.table_id)
    

table_cnt = len(chicago_dataset_tables)
print('tables cnt: {0}'.format(table_cnt))

'crime'

tables cnt: 1


In [23]:
crime_table_ref = chicago_dataset_ref.table('crime')
crime_table = client.get_table(crime_table_ref)

In [30]:
columns_cnt = len(crime_table.schema)
display(columns_cnt)

22

In [44]:
coordinates_df = client.list_rows(crime_table, selected_fields=crime_table.schema[-3:-1], max_results=500).to_dataframe()
display(coordinates_df.head())

,latitude,longitude
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,41.807911,-87.720354
4,41.969135,-87.798904


In [45]:
openaq_dataset_ref = client.dataset('openaq', project='bigquery-public-data')
openaq_dataset = client.get_dataset(openaq_dataset_ref)

openaq_tables = list(client.list_tables(openaq_dataset))
# dbg
for table in openaq_tables:
    print(table.table_id)

global_air_quality


In [46]:
openaq_table_ref = openaq_dataset_ref.table('global_air_quality')
openaq_table = client.get_table(openaq_table_ref)

In [47]:
# dbg
display(
    client.list_rows(openaq_table, max_results=5).to_dataframe()
)

,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours
0,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,co,910.00,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
1,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,no2,131.87,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
2,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,o3,15.57,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
3,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,pm25,45.62,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
4,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,so2,4.49,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25


In [48]:
# Try out select query
# note that ` in FROM part is not ' or " character

us_cities_select_query = \
    """
    SELECT city
    FROM `bigquery-public-data.openaq.global_air_quality`
    WHERE country='US'
    """
query_job = client.query(us_cities_select_query)
us_cities_df = query_job.to_dataframe()

In [55]:
# dbg
us_cities_df.info()

display(
    us_cities_df['city'].value_counts().head(3)
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3610 entries, 0 to 3609
Data columns (total 1 columns):
city    3610 non-null object
dtypes: object(1)
memory usage: 28.3+ KB


Phoenix-Mesa-Scottsdale             87
Houston                             78
Los Angeles-Long Beach-Santa Ana    60
Name: city, dtype: int64

In [56]:
# Note: always check limits when executing queries on huge datasets!
# Example of a big dataset: hacker_news

# Workaround: QueryJobConfig with dry_run=True
# This one will estimate size of query without running it

hackernews_select_query = \
    """
    SELECT score, title
    FROM `bigquery-public-data.hacker_news.full`
    WHERE type = "job";
    """
dry_run_config = bigquery.QueryJobConfig(dry_run=True)

In [57]:
# Estimate costs with dry run
dry_run_query_job = client.query(
    hackernews_select_query,
    job_config=dry_run_config
)

print('Estimated {0} bytes when processing the query'.format(
    dry_run_query_job.total_bytes_processed
))

Estimated 371762945 bytse when processing the query


In [58]:
# Limit how much data I am willing to scan:

BYTES_1_MB = 1024 * 1024
safe_config = bigquery.QueryJobConfig(
    maximum_bytes_billed=BYTES_1_MB
)

safe_query_job = client.query(
    hackernews_select_query,
    job_config=safe_config
)
hacker_news_1_mb = safe_query_job.to_dataframe()

In [63]:
# Cancelled query: 1 MB limit was exceeded

# # Only run the query if it's less than 1 MB
# ONE_MB = 1000*1000
# safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=ONE_MB)

# # Set up the query (will only run if it's less than 1 MB)
# safe_query_job = client.query(hackernews_select_query, job_config=safe_config)

# # API request - try to run the query, and return a pandas DataFrame
# safe_query_job.to_dataframe()

In [67]:
ppm_countries_select_query = \
    """
    SELECT DISTINCT country
    FROM `bigquery-public-data.openaq.global_air_quality`
    WHERE unit = "ppm";
    """
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=1024*1024*10)
countries_query_job = client.query(
    ppm_countries_select_query,
    job_config=safe_config
)

In [68]:
ppm_countries_results_df = countries_query_job.to_dataframe()
display(ppm_countries_results_df)

,country
0,US
1,CL
2,MX
3,BA
4,CA
5,IL
6,TW
7,CO
8,TH
9,BR


In [71]:
# parent -> comment that was replied to
# id -> unique ID of each comment

popular_comments_select_query = \
    """
    SELECT
        parent AS comment_id,
        COUNT(id) AS comment_replies_cnt
    FROM 
        `bigquery-public-data.hacker_news.comments`
    GROUP BY parent
    HAVING COUNT(id) > 0;
    """
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=1024*1024*10)
query_job = client.query(popular_comments_select_query, job_config=safe_config)

popular_comments_df = query_job.to_dataframe()
display(popular_comments_df.head())

In [ ]:
prolific_commenters_select_query = \
    """
    SELECT
        author AS author_id,
        COUNT(id) AS comments_cnt
    FROM
        `bigquery-public-data.hacker_news.comments`
    GROUP BY id
    HAVING COUNT(author) > 10000;
    """

In [ ]:
dow_fatal_accidents_cnt_select_query = \
    """
    SELECT
        COUNT(consecutive_number) AS fatal_accidents_cnt,
        EXTRACT(DAYOFWEEK FROM timestamp_of_crash) AS dow
    FROM
        `bigquery-public-data.nhtsa_traffic_fatalities.accident_2015`
    GROUP BY dow
    ORDER BY fatal_accidents_cnt DESC;
    """

In [74]:
crypto_dataset_ref = client.dataset('crypto_bitcoin', project='bigquery-public-data')
dataset = client.get_dataset(crypto_dataset_ref)

transactions_table_ref = crypto_dataset_ref.table('transactions')
transactions_table = client.get_table(transactions_table_ref)

In [75]:
transactions_preview_df = client.list_rows(
    transactions_table,
    max_results=5
).to_dataframe()

display(transactions_preview_df)

,hash,size,virtual_size,version,lock_time,block_hash,block_number,block_timestamp,block_timestamp_month,input_count,output_count,input_value,output_value,is_coinbase,fee,inputs,outputs
0,aaac8b454e079237b8509c422b8a0ebd95e3eca5944b95...,137,137,1,0,000000000000000000d6cdda66edb65e8969566ca65fe3...,484015,2017-09-07 13:43:30+00:00,2017-09-01,0,1,None,1266641901,True,0,[],"[{'index': 0, 'script_asm': 'OP_DUP OP_HASH160..."
1,8505c4277a8da0450cb6fd345b409a87218e09987c7e92...,193,166,1,0,000000000000000000427219aa8b931acb167f582e9b9c...,485083,2017-09-13 19:30:26+00:00,2017-09-01,0,2,None,1367011480,True,0,[],"[{'index': 0, 'script_asm': 'OP_HASH160 6dffc0..."
2,d1a2060a08e8cd54480de8a0aac761b8667c175a7de08e...,213,186,1,0,000000000000000000d5ed15beb3085719e8f1b95f2924...,485005,2017-09-13 10:25:46+00:00,2017-09-01,0,2,None,1255484937,True,0,[],"[{'index': 0, 'script_asm': 'OP_DUP OP_HASH160..."
3,2fba35aaba056d4028d512c0fb134ff41d2d5c504fc7a1...,213,186,1,0,0000000000000000009fd230518a5b3d50ebcd5120a844...,486934,2017-09-25 17:28:19+00:00,2017-09-01,0,2,None,1431309437,True,0,[],"[{'index': 0, 'script_asm': 'OP_DUP OP_HASH160..."
4,630af30aa1ade3de86df071d35b6eb28e0590cf7e0bcaa...,217,190,1,0,000000000000000000841e1fa84e8e5abdfa000936524c...,483854,2017-09-06 16:06:09+00:00,2017-09-01,0,2,None,1530785512,True,0,[],"[{'index': 0, 'script_asm': 'OP_DUP OP_HASH160..."
